# V. Application de la Quantification

In [ ]:
%cd /kaggle/working
!git clone --depth=1 https://github.com/ggerganov/llama.cpp.git
%cd /kaggle/working/llama.cpp
!sed -i 's|MK_LDFLAGS   += -lcuda|MK_LDFLAGS   += -L/usr/local/nvidia/lib64 -lcuda|' Makefile
!LLAMA_CUDA=1 conda run -n base make -j > /dev/null

In [ ]:
!ls /kaggle/working/

In [ ]:
# Appliquer le script de quantization
!/kaggle/working/llama.cpp/llama-quantize /kaggle/input/fine-tuning-chatdoctor-4/llama-3-8b-chat-doctor-merged.gguf /kaggle/working/llama-3-8b-chat-doctor-merged-Q4_K_M.gguf Q4_K_M

In [ ]:
!ls /kaggle/working/

In [ ]:
# Pousser le modèle quantifier sur Hugging Face ppur ensuite pouvoir l'utiliser sur LM Studio. Si on souhaite l'utiliser sur JAN, il faut le télécharger sur l'ordinateur directement. 
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HF ")
login(token = hf_token)

In [ ]:
from huggingface_hub import HfApi, create_repo, upload_file

# Identifiant du dépôt
repo_id = "PierreMaxime/llama-3-8b-chat-doctor-merged-Q4_K_M"

# Initialisez l'API
api = HfApi()

# Créez le dépôt
api.create_repo(repo_id=repo_id, private=False)  # Utilisez private=True si vous voulez un dépôt privé

# Chemin vers le fichier modèle quantisé à uploader
quantized_file_path = "/kaggle/working/llama-3-8b-chat-doctor-merged-Q4_K_M.gguf"

# Pousser le fichier modèle quantisé sur Hugging Face
upload_file(
    path_or_fileobj=quantized_file_path,
    path_in_repo="llama-3-8b-chat-doctor-merged-Q4_K_M.gguf",  # Nom du fichier dans le dépôt
    repo_id=repo_id,
    commit_message="Ajout du modèle quantisé",
    token=hf_token
)
